# Movie Ratings Capstome

#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import prepare

import warnings
warnings.filterwarnings('ignore')

### Get Raw dirty data https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=links.csv

In [2]:
raw = prepare.get_dirty_data()

Using cached CSV


In [9]:
raw.head(2)

,Unnamed: 0,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,0,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter JossWhedon AndrewStanton JoelCohe...,862,Toy Story,Animation Comedy Family,30000000,"Led by Woody, Andy's toys live happily in his ...",21.946943,PixarAnimationStudios,UnitedStatesofAmerica,373554033.0,81.0,7.7,5415.0,jealousy toy boy friendship friends rivalry bo...,1995-10-30
1,1,RobinWilliams JonathanHyde KirstenDunst Bradle...,LarryJ.Franco JonathanHensleigh JamesHorner Jo...,8844,Jumanji,Adventure Fantasy Family,65000000,When siblings Judy and Peter discover an encha...,17.015539,TriStarPictures TeitlerFilm InterscopeCommunic...,UnitedStatesofAmerica,262797249.0,104.0,6.9,2413.0,board game disappearance based on children's b...,1995-12-15


#### Initial Data Check

In [12]:
# Nulls by Column

prepare.nulls_by_columns(raw)

,count,percent
Unnamed: 0,0,0.000000
cast,34,0.004559
crew,13,0.001743
id,0,0.000000
title,0,0.000000
genres,23,0.003084
budget,0,0.000000
overview,22,0.002950
popularity,0,0.000000
production_companies,418,0.056055


In [14]:
# nulls by Rows

prepare.nulls_by_rows(raw)

n_missing  percent_missing
0          0.000000           6502
1          0.058824            684
2          0.117647            175
3          0.176471             73
4          0.235294             16
5          0.294118              3
6          0.352941              4
dtype: int64

In [ ]:
# Missing Values 



### Prepare Function Implements:
- Fill nulls on columns by ____ 
- Drop duplicates
- 
- 


# Prepare Data

> ### Get prepared data
> - Handle missing values
> - 
> - 

> ### Split the data

# Explore Data

> ### Use train set

> ### Key exploration take-aways

# Model Data



In [ ]:
# 

### Initial Explorations

> #### Check summary statictics

In [4]:
prepare.data_summary(raw)

--- Shape: (7457, 17)
--- Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7457 entries, 0 to 7456
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            7457 non-null   int64  
 1   cast                  7423 non-null   object 
 2   crew                  7444 non-null   object 
 3   id                    7457 non-null   int64  
 4   title                 7457 non-null   object 
 5   genres                7434 non-null   object 
 6   budget                7457 non-null   int64  
 7   overview              7435 non-null   object 
 8   popularity            7457 non-null   float64
 9   production_companies  7039 non-null   object 
 10  production_countries  7301 non-null   object 
 11  revenue               7457 non-null   float64
 12  runtime               7451 non-null   float64
 13  vote_average          7457 non-null   float64
 14  vote_count            7457 non-null   flo

In [15]:
name_basic.shape

(11654331, 6)

In [16]:
titles_basics.head(1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"


In [17]:
titles_basics.shape

(8946058, 9)

In [19]:
titles_crew.head(1)

,tconst,directors,writers
0,tt0000001,nm0005690,\N


In [20]:
titles_crew.shape

(8946058, 3)

In [21]:
titles_principals.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"


In [22]:
titles_principals.shape

(50453291, 6)

In [33]:
titles_akas.head(1)

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0


In [24]:
titles_akas.shape

(32079668, 8)

### Merge the tables

- __join='outer'__ - 0 information loss, all union

- __join='inner'__ - intersection 

- __one-to-one joins:__ for example when joining two DataFrame objects on their indexes (which must contain unique values).

- __many-to-one joins:__ for example when joining an index (unique) to one or more columns in a different DataFrame.

- __many-to-many joins:__ joining columns on columns.

In [25]:
# Rename columns in akas set

titles_akas = titles_akas.rename(columns = {'titleId': 'tconst'})

In [31]:
# Reset index on titles_basics

titles_basics = titles_basics.reset_index(drop = True) 

In [38]:
titles_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


> #### (A). Merge titles_basic with titles_akas (on tconst)

In [50]:
akas_titles_basics = pd.merge(titles_basics,titles_akas, on = 'tconst' , sort=False)
akas_titles_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5,Карменсита,RU,\N,imdbDisplay,\N,0


> #### (B). Merge titles_basic, titles_akas with title_ratings (on tconst)

In [53]:
ratings_akas_titles_basics= pd.merge(akas_titles_basics, titles_ratings, on = 'tconst' , sort=False)
ratings_akas_titles_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,title,region,language,types,attributes,isOriginalTitle,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1,Карменсіта,UA,\N,imdbDisplay,\N,0,5.7,1882
1,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",2,Carmencita,DE,\N,\N,literal title,0,5.7,1882
2,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0,5.7,1882
3,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0,5.7,1882
4,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5,Карменсита,RU,\N,imdbDisplay,\N,0,5.7,1882


> #### (C). Merge titles_basic, titles_akas, title ratings with titles_principals (on tconst)

In [55]:
ratings_akas_titles_basics_principals = pd.merge(ratings_akas_titles_basics, titles_principals, on = 'tconst' , sort=False)
ratings_akas_titles_basics_principals.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering_x,...,types,attributes,isOriginalTitle,averageRating,numVotes,ordering_y,nconst,category,job,characters
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1,...,imdbDisplay,\N,0,5.7,1882,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1,...,imdbDisplay,\N,0,5.7,1882,2,nm0005690,director,\N,\N
2,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1,...,imdbDisplay,\N,0,5.7,1882,3,nm0374658,cinematographer,director of photography,\N
3,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",2,...,\N,literal title,0,5.7,1882,1,nm1588970,self,\N,"[""Self""]"
4,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",2,...,\N,literal title,0,5.7,1882,2,nm0005690,director,\N,\N


> #### (D). Merge titles_basic, titles_akas, title ratings, titles_principals with _________ (on tconst)

In [ ]:
# # Join all tables 

# (titles_basics.set_index('tconst')\
#             .join(titles_akas.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
#             .join(titles_crew.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
#             .join(titles_principals.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
#             .join(titles_ratings.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
# )